In [1]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

!pip install -U adapter-transformers
!pip install datasets

     |████████████████████████████████| 2.9 MB 4.1 MB/s 
     |████████████████████████████████| 61 kB 555 kB/s 
     |████████████████████████████████| 596 kB 58.2 MB/s 
     |████████████████████████████████| 895 kB 38.1 MB/s 
     |████████████████████████████████| 3.3 MB 55.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 298 kB 4.1 MB/s 
     |████████████████████████████████| 243 kB 66.9 MB/s 
     |████████████████████████████████| 1.1 MB 61.4 MB/s 
     |████████████████████████████████| 132 kB 84.0 MB/s 
     |████████████████████████████████| 271 kB 55.7 MB/s 
     |████████████████████████████████| 160 kB 86.5 MB/s 
     |████████████████████████████████| 192 kB 76.1 MB/s 


In [2]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("yxchar/chemprot-tlm")
dataset.num_rows

Using custom data configuration yxchar___chemprot-tlm-5c863857db86ab7c


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/955k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/823k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/581k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/yxchar___chemprot-tlm-5c863857db86ab7c/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'test': 3469, 'train': 4169, 'validation': 2427}

In [3]:
dataset["train"][0]

{'Unnamed: 0': 0,
 'id': 0,
 'label': 0,
 'text': '<< Epidermal growth factor receptor >> inhibitors currently under investigation include the small molecules [[ gefitinib ]] (Iressa, ZD1839) and erlotinib (Tarceva, OSI-774), as well as monoclonal antibodies such as cetuximab (IMC-225, Erbitux).'}

In [4]:
!pip3 install torch torchvision

In [5]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=512, truncation=True, padding="max_length")

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset.rename_column_("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: rename_column_ is deprecated and will be removed in the next major version of datasets. Use DatasetDict.rename_column instead.
  if sys.path[0] == '':


In [6]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=13,
)
model = RobertaModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

In [7]:
# Add a new adapter
model.add_adapter("chemprot")
# Add a matching classification head
model.add_classification_head(
    "chemprot",
    num_labels=13
    # id2label={ 0: "👎", 1: "👍"}
  )
# Activate the adapter
model.train_adapter("chemprot")

In [8]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=5e-5,
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

In [9]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec 15 01:53:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    32W / 250W |   1653MiB / 16280MiB |     17%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
trainer.train()

***** Running training *****
  Num examples = 4169
  Num Epochs = 12
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3132


Step,Training Loss
200,2.032300
400,1.500400
600,1.066400
800,0.864700
1000,0.752400
1200,0.666900
1400,0.641300
1600,0.598800
1800,0.563900
2000,0.535600


Saving model checkpoint to ./training_output/checkpoint-500
Configuration saved in ./training_output/checkpoint-500/chemprot/adapter_config.json
Module weights saved in ./training_output/checkpoint-500/chemprot/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-500/chemprot/head_config.json
Module weights saved in ./training_output/checkpoint-500/chemprot/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-500/chemprot/head_config.json
Module weights saved in ./training_output/checkpoint-500/chemprot/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-500/chemprot/head_config.json
Module weights saved in ./training_output/checkpoint-500/chemprot/pytorch_model_head.bin
Saving model checkpoint to ./training_output/checkpoint-1000
Configuration saved in ./training_output/checkpoint-1000/chemprot/adapter_config.json
Module weights saved in ./training_output/checkpoint-1000/chemprot/pytorch_adapter.bin
Configuration saved in

TrainOutput(global_step=3132, training_loss=0.7687966162826518, metrics={'train_runtime': 1994.3396, 'train_samples_per_second': 25.085, 'train_steps_per_second': 1.57, 'total_flos': 1.3392462083641344e+16, 'train_loss': 0.7687966162826518, 'epoch': 12.0})

In [10]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2427
  Batch size = 16


{'epoch': 12.0,
 'eval_acc': 0.8121137206427689,
 'eval_loss': 0.6349377632141113,
 'eval_runtime': 47.2158,
 'eval_samples_per_second': 51.402,
 'eval_steps_per_second': 3.219}

In [11]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

classifier("This is awesome!")

[{'label': 'LABEL_0', 'score': 0.21145081520080566}]

In [12]:
model.save_adapter("./final_adapter", "chemprot")

!ls -lh final_adapter

Configuration saved in ./final_adapter/adapter_config.json
Module weights saved in ./final_adapter/pytorch_adapter.bin
Configuration saved in ./final_adapter/head_config.json
Module weights saved in ./final_adapter/pytorch_model_head.bin


total 5.8M
-rw-r--r-- 1 root root  574 Dec 15 03:23 adapter_config.json
-rw-r--r-- 1 root root  607 Dec 15 03:23 head_config.json
-rw-r--r-- 1 root root 3.5M Dec 15 03:23 pytorch_adapter.bin
-rw-r--r-- 1 root root 2.3M Dec 15 03:23 pytorch_model_head.bin


In [13]:
from google.colab import files
!zip -r /content/file.zip /content
files.download("/content/file.zip")

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2021.12.03/ (stored 0%)
  adding: content/.config/logs/2021.12.03/14.33.37.701606.log (deflated 53%)
  adding: content/.config/logs/2021.12.03/14.33.36.903459.log (deflated 54%)
  adding: content/.config/logs/2021.12.03/14.32.30.027140.log (deflated 91%)
  adding: content/.config/logs/2021.12.03/14.32.50.522723.log (deflated 53%)
  adding: content/.config/logs/2021.12.03/14.33.09.955489.log (deflated 86%)
  adding: content/.config/logs/2021.12.03/14.33.16.964195.log (deflated 54%)
  adding: content/.config/.last_update_check.json (deflated 24%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configuration

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
pred, label, metrics = trainer.predict(dataset["test"])
pred_label = np.argmax(pred, axis = 1)

from sklearn.metrics import classification_report 
print(classification_report(label, pred_label))

***** Running Prediction *****
  Num examples = 3469
  Batch size = 16


              precision    recall  f1-score   support

           0       0.89      0.92      0.91      1255
           1       0.91      0.85      0.88       293
           2       0.75      0.90      0.81       182
           3       0.52      0.18      0.27        72
           4       0.65      0.60      0.63       191
           5       0.81      0.78      0.80       453
           6       0.75      0.69      0.72       334
           7       0.00      0.00      0.00        41
           8       0.74      0.84      0.79       340
           9       0.71      0.85      0.77       292
          10       0.00      0.00      0.00         4
          11       0.00      0.00      0.00        12

    accuracy                           0.81      3469
   macro avg       0.56      0.55      0.55      3469
weighted avg       0.80      0.81      0.80      3469



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
